# Cognite workshop lab 2 - Time series and data points
Estimated time: 30 minutes.

## Suggested pre-read
Before you begin, you should read about time series and datapoints in the documentation: https://docs.cognite.com/dev/concepts/resource_types/timeseries.html

## Imports and Cognite SDK client
As usual, we need to import some modules and create a Cognite SDK client.

In [ ]:
from getpass import getpass
from cognite.client import CogniteClient

In [ ]:
client = CogniteClient(
    api_key=getpass("Please enter API key: "),
    project="publicdata", 
    client_name="DSHub", 
    base_url="https://api.cognitedata.com"
)

## Verify login
Retrieve some assets to verify that we have access to the project.

In [ ]:
assets = client.assets.list()
assets

## Create a time series and attach it to an asset
The time series object does not contain data points itself, but is the metadata object with properties such as `name`, `unit` and `description`. So in order to create data points, we first need to have a time series. While we're at it, let's also create an asset so we can attach the time series to that asset.

As before, we need to import the data classes we want to create. We start by creating the asset we want to attach the time series to.

SDK documentation: https://cognite-docs.readthedocs-hosted.com/projects/cognite-sdk-python/en/latest/cognite.html#create-assets

(If you get an error message with error code 403, it means you don't have access to create assets. Head over to [console](https://console.cognitedata.com) and create a group with write access to assets, and assign that group to your service account.)

In [ ]:
from cognite.client.data_classes import Asset
asset = Asset(name="MyAsset", source="dshub_workshop")
created_asset = client.assets.create(asset)
created_asset

Now that we created this asset, we see that it has an `id`, a unique identifier that we can use to refer to that asset. We will now we create the time series.

SDK documentation: https://cognite-docs.readthedocs-hosted.com/projects/cognite-sdk-python/en/latest/cognite.html#create-time-series

(If you get an error message with error code 403, it means you don't have access to create time series. Head over to [console](https://console.cognitedata.com) and create a group with write access to time series, and assign that group to your service account.)

In [ ]:
from cognite.client.data_classes import TimeSeries
client.time_series.delete(id=created_time_series.id)
time_series = TimeSeries(name="MyTimeSeries", asset_id=created_asset.id)
created_time_series = client.time_series.create(time_series)
created_time_series

Congratulations! You have now also created a time series. Not only that, but it is also attached to the asset (you can verify that by comparing the `assetId` field to the id of the asset you created above). Next step is to create some data points on the time series.

## Create data points
Data points are values at a certain time. Typical examples are temperature, pressure or speed as a function of time. In this tutorial, we will put in a simple sine wave `sin(wt)`. This can of course be done using less code, but the intent here is readability for most people.

SDK documentation: https://cognite-docs.readthedocs-hosted.com/projects/cognite-sdk-python/en/latest/cognite.html#insert-data-points

In [ ]:
import time
import datetime
import numpy as np

f = 1.0 / (24 * 60 * 60 * 1000) # 1.0 / number_of_ms_per_day
w = 2 * np.pi * f # angular frequency

# time is always measured in the UTC time zone
now = datetime.datetime.utcnow()

# we will store the data points in this list
timestamps = [] # this will contain all timestamps
values = [] # this will contain all values

minutes_in_30_days = 30 * 24 * 60
for minute in range(0, minutes_in_30_days):
    # calculate sin(wt) for each minute over the past 30 days
    date = (now - datetime.timedelta(minutes=minute))
    # convert to unix timestamp in miliseconds as that's what API wants
    timestamp_ms = time.mktime(date.timetuple()) * 1000 # ms
    # short hand variable so it looks more like the mathematical expression
    t = timestamp_ms 
    # this is the time series value
    y = np.sin(w * t) 
    # append the values to separate lists
    timestamps.append(t)
    values.append(y)

datapoints = list(zip(timestamps, values)) # convert the two lists into [ (t1, v1), (t2,v2), ...]
client.datapoints.insert(datapoints, id=created_time_series.id)
print(f"Inserted {len(datapoints)} data points into the time series.")

Congratulations, you have now inserted about 43000 data points to your time series. We will now retrieve them to plot and see the result.

## Plot the data
The simplest way to plot any data is to just use `client.time_series.retrieve(id=<time-series-id>)`, which by default will contain all the data points over the past 24 hours. That object has a function called `plot` which does exactly what you'd expect. Let's do that first.

In [ ]:
my_time_series = client.time_series.retrieve(id=created_time_series.id)
my_time_series.plot()

If you want to see more data, the `plot` function accepts a range of parameters where you control the time interval, and which aggregate and granularity you want. The granularity lets you choose what resolution you want. If you have sensor values coming in every second, and you look at the data over the past 10 years, you probably don't want all those 315360000 data points (that corresponds to about 5 GB of data). You might want to choose granularity of `1d`.

When you choose a lower resolution than your data points, that means the value you get needs to be computed from all the underlying values. Let's say you have data points every minute, but you want a granularity of 1 hour. That means that for each data point (every hour), you have 60 data points (every minute) that will be used to produce that one value. CDF allows different choices here, but we will use the `average` value. You can also get `max` and `min` within each interval. See https://docs.cognite.com/api/v1/#operation/getMultiTimeSeriesDatapoints for valid aggregate functions and granularities.

We will now plot data over the past two weeks with a granularity of one hour. You can try to change these values to see the effect.

In [ ]:
my_time_series.plot(start="2w-ago", end="now", aggregates=["average"], granularity="1h")

## Retrieve data points
If you want to retrieve data points to analyze them, or plot them in a different way, you can retrieve them using `client.datapoints.retrieve(id=<time-series-id>)`. This function also accepts the same choices for time interval, granularity and aggregates. Try choosing different values and see how the number of retrieved data points varies.

In [ ]:
my_datapoints = client.datapoints.retrieve(id=created_time_series.id, start="2w-ago", end="now", aggregates=["average"], granularity="1h")
print(f"Retrieved {len(my_datapoints)} data points")

## Cleanup - delete tutorial data
We now delete the objects we created above. Since we have the id's we can just delete them directly

In [ ]:
client.assets.delete(id=created_asset.id)
client.time_series.delete(id=created_time_series.id)

Congratulations, you have now learned how to create a time series, attach it to an asset. With that time series, you have inserted data points, retrieved them and plotted them. In the next tutorial, you will learn how to create assets and time series when data comes from `csv` files. This happens quite often at customers.